# Approximation of N-body matrix by MCBH method

Setting of the problem: for given particles $X_1, \ldots, X_N$ calculate potential $f(x, y) = -\log \vert x-y \vert$ they create on themselves.

Since goal of this example is to show how to use ***h2tools*** package, we only approximate following matrix:


$$
A_{ij} = \left\{ \begin{array}{cl} -\log{\vert X_i-X_j\vert}, & i \ne j \\ 0, & i = j \end{array} \right.
$$

At first, we import all necessary modules, generate data and set variables.
Then we build cluster trees, approximate matrix with MCBH algorithm and measure relative error of far field approximation.

In [1]:
# main imports and variables

# make this script work with both Python 2 and Python 3
from __future__ import print_function, absolute_import, division

# import all necessary modules and set variables
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
# at first, import predefined functions and dataclass for particles from h2py.collections
from h2tools.collections import particles
# import main container for problem and cluster tree
from h2tools.problem import Problem, ClusterTree
# import multicharge approximation method
from h2tools.mcbh import mcbh

import numpy as np

# set dimensionality of the problem (ndim), size of the problem (count),
# desired relative accuracy of far field approximation (tau),
# number of iterations for MCBH algorithm (iters),
# whether to hold special submatrices in memory or compute them on demand (onfly),
# symmetricity of the problem (symmetric),
# maximum size of leaf node of cluster trees (block_size) and
# verbosity (verbose)
ndim = 2
count = 5000
tau = 1e-5
iters = 2
onfly = 0
symmetric = 1
block_size = 20
verbose = 1

func = particles.log_distance

In [2]:
#generate data
# set random seed so that results are repeatable
np.random.seed(0)
# generate positions of particles in shape (ndim, count)
position = np.random.randn(ndim, count)
# create data object from given positions of particles
data = particles.Particles(ndim, count, position)
# initialize cluster tree from data object
tree = ClusterTree(data, block_size)
# create main problem object with interaction particles.inv_distance
problem = Problem(func, tree, tree, symmetric, verbose)

Cluster trees are generated in 0.298183917999 seconds
Depth level of each cluster tree: 11
Row cluster tree
    nodes : 739
    leaves: 370
Column cluster tree
    nodes : 739
    leaves: 370


In [3]:
# import multicharge approximation method
from h2tools.mcbh import mcbh

# build approximation of the matrix with MCBH algorithm

matrix = mcbh(problem, tau, iters=iters, onfly=onfly, verbose=verbose)

# check approximation error
print(matrix.diffnorm(far_only=1))

# Compress matrix
matrix.svdcompress(1e-4, verbose=1)
print(matrix.diffnorm(far_only=1))

matrix.svdcompress(1e-3, verbose=1)
print(matrix.diffnorm(far_only=1))

Far-field interactions(MCBH method):
    Function calls: 7796
    Function values computed: 13122651
    Function time, seconds: 0.41
    Average time per function value, seconds: 3.10e-08
    Maxvol time, seconds: 2.0126247406
Near-field interactions:
    Function calls: 1551
    Function values computed: 404438
    Function time, seconds: 0.03
    Average time per function value, seconds: 8.04e-08
Total time, seconds: 2.70
Memory:
    Basises, MB: 0.14
    Transfer matrices, MB: 1.26
    Far-field interactions, MB: 5.52
    Near-field interactions, MB: 3.46
Total memory, MB: 10.38
1.13538230441e-05
memory BEFORE SVD-compression: 10.38MB
memory AFTER SVD-compression: 5.46MB
recompression time: 1.17842912674
0.000141407139373
memory BEFORE SVD-compression: 5.46MB
memory AFTER SVD-compression: 5.06MB
recompression time: 1.17794513702
0.00144608894798


In [4]:
print(matrix)

H2matrix at 0x1084b8690
    Structure (h2/mcbh): h2
    Memory layout (low/full): full
    Shape: [5000, 5000]
    Total memory, MB: 5.06
        Basises, MB: 0.01
        Transfer matrices, MB: 0.56
        Far-field interactions, MB: 1.03
        Near-field interactions, MB: 3.46
